In [1]:
import pandas as pd
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import time

# NASA Mars News

In [2]:
#Mars News url
news_url = 'https://mars.nasa.gov/news'

#Browser setup
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome')
browser.visit(news_url)

In [ ]:
#Create BeautifulSoup object and parse
soup = bs(browser.html, 'html.parser')

In [ ]:
#Latest news title
news_title = soup.find(class_='content_title').get_text(strip=True)
news_title

In [ ]:
#Paragraph from title
news_p = soup.find(class_='article_teaser_body').get_text(strip=True)
news_p

# JPL Mars Space Images - Featured Image

In [16]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
base_url ="https://www.jpl.nasa.gov"

In [17]:
#Browser setup
browser.visit(jpl_url)

In [18]:
#Utilize Browser objects to manipulate website
browser.click_link_by_id("full_image")
time.sleep(3)
#Utilize Browser objects to manipulate website
browser.click_link_by_partial_text('more info')

In [19]:
#Create BeautifulSoup and parse;
html = browser.html
featured_image_soup = bs(html,"html.parser")

In [20]:
#Obtain image href, alt description, and connect to url
featured_image = featured_image_soup.find(class_="lede")
featured_image_alt = featured_image.a.img['alt']
featured_image_url = (base_url + featured_image.a["href"])

In [21]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18826_hires.jpg


# Mars Weather

In [ ]:
tweeter_url = "https://twitter.com/marswxreport?lang=en"

In [ ]:
#Browser setup
browser.visit(tweeter_url)
#Create BeautifulSoup and parse;
html = browser.html
tweet_soup = bs(html, 'html.parser')
#Obtain and asign mars_weather result
mars_weather = tweet_soup.find(class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').get_text(strip=True)

In [ ]:
print(mars_weather)

# Mars Facts

In [12]:
def planet_facts(planet='mars'):
    '''
    Scrap data from NASA website regarding information about any of the 8 planets.
    USE: mercury, venus, earth, mars, jupiter, saturn, uranus, neptune
    
    *Use lower case*
    *Default value is mars*
    '''
    try:
        planet_facts_url = ("https://space-facts.com/" + planet + "/")
        #Browser setup
        browser.visit(planet_facts_url)
        data = pd.read_html(planet_facts_url)
        data = data[0]
        data.columns = ['Description', 'Info']
        data.to_html(index=False)
        data = data.replace('\n','')
        
        return data
    except Exception as e:
        print('Please read docstring')

# Mars Hemispheres

In [3]:
mars_hem = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
base_url = "https://astrogeology.usgs.gov/"

In [4]:
browser.visit(mars_hem)

In [5]:
html = browser.html
mars_hem_soup = bs(html, 'html.parser')

In [6]:
mars_data = mars_hem_soup.find_all(class_='itemLink product-item')
mars_data[2]['href']

'/search/map/Mars/Viking/schiaparelli_enhanced'

In [ ]:
#Generate empty lists
hemisphere_image_urls = []
hemisphere_dict = {}

for link in range(0,len(mars_data),2):
    #Visit each hemisphere url
    browser.visit(base_url + mars_data[link]["href"])
    time.sleep(3)
    
    #Convert page to html to parse title
    html = browser.html
    mars_hem_soup = bs(html, "html.parser")
    
    #Scrape for text
    #Extract title from h2 tag
    hemisphere_title = mars_hem_soup.find("h2", class_="title").text
     
    #Look for 'sample' button that contains our image links
    sample_button = browser.find_link_by_text("Sample").first
    
    #Save image links to our list
    hemisphere_dict = {"title": hemisphere_title, "img_url": sample_button["href"]}
    hemisphere_image_urls.append(hemisphere_dict)

#Close open browser
browser.quit()

In [ ]:
hemisphere_image_urls